<a href="https://colab.research.google.com/github/zilioalberto/Machine_Learning_ANN/blob/main/ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## Alunos:
## Alberto Zilio
## Roni Pereira